## 일급 함수
* ### 런타임에 생성할 수 있다
* ### 데이터 구조체의 변수나 요소에 할당할 수 있다
* ### 함수 인수로 전달할 수 있다
* ### 함수 결과로 반환할 수 있다

In [4]:
def factorial(n):
    '''returns n'''
    return 1 if n<2 else n * factorial(n-1)

fact = factorial # 함수를 호출한 것이 아닌 함수를 fact 변수에 할당
fact # <function __main__.factorial(n)>
fact(5) # 120

map(factorial,range(11)) # <map at 0x16b851abc40> -> factorial 함수를 0부터 10까지의 숫자에 적용한 결과를 반환하는 map 객체를 생성합니다. map 객체는 이터레이터이므로, 직접 출력하면 메모리 주소가 나타납니다.


list(map(fact,range(11)))
# list(map(fact, range(11)))는 0부터 10까지의 각 숫자에 대해 factorial 함수를 적용한 결과를 리스트로 변환하여 [1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]을 반환합니다.

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## 고위 함수
* ### 함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 고위함수 ex) map함수, sort함수


In [7]:
fruits = ['strawberry','fig','apple','cherry','raspberry','banana']
sorted(fruits,key=len)

# 람다를 이용한 역순정렬
sorted(fruits,key=lambda word:word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [ ]:
import random

class BingoCage:
    def __init__(self,items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()
    
    
bingo = BingoCage(range(3))
bingo()
bingo()
bingo() 


## 일급 함수 디자인 패턴
* ### 일급 함수를 지원하는 언어에서는 전략,명령,템플릿 메서드,비지터 패턴을 눈여겨 봐야함

In [ ]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion

    def total(self):
        if not hasattr(self,'_total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{: 2f}>'
        return fmt.format(self.total(),self.due())

class Promotion(ABC): # 전략:추상 베이스 클래스
    @abstractmethod
    def discount(self,order):
        """할인액을 구체적인 숫자로 반환한다"""

class FidelityPromo(Promotion): # 전략객체
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self,order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self,order): # 전략객체
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

class LargeOrderPromo(Promotion): # 전략객체
    """10종류 이상의 상품을 구입하면 10% 할인 적용"""

    def discount(self,order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total()* .07
        return 0

joe = Customer('John Doe',0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple',10, 1.5),
        LineItem('watermellon',5,5.0)]
o1 = Order(joe,cart,FidelityPromo())
o2 = Order(ann,cart,FidelityPromo())
print(o1,o2)

* ### 함수지향 전략
* ### 전략 객체는 상태(객체 속성)를 가지고 있지 않음 
* ### 플라이웨이트 : 여러 콘텍스트에서 동시에 사용할 수 있는 공유 객체
* ### 함수는 파이썬이 모듈을 컴파일할 떄 단 한번만 생성되므로 플라이웨이트가 필요없음
* ### 일반 함수도 '여러 콘텍스트에서 동시에 공유할 수 있는 공유 객체'임

In [ ]:
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    def __init__(self,product,quantity,price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity

class Order:  # Context
    def __init__(self,customer,cart,promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion # 할인 전략 주입

    def total(self):
        if not hasattr(self,'_total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due:{: 2f}>'
        return fmt.format(self.total(),self.due())

def fidelity_promo(order): # 전략객체
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def LargeOrderPromo(order): # 전략객체
    """10종류 이상의 상품을 구입하면 10% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()* .07
    return 0